<a href="https://colab.research.google.com/github/sallywang147/SCInvarinfer/blob/main/%5BEvaluation%5D_llama_for_smart_contracts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install --quiet accelerate
!pip install -i --quiet https://test.pypi.org/simple/ bitsandbytes
!pip install --quiet sentencepiece
!pip install --quiet git+https://github.com/huggingface/transformers
!pip install --quiet rich[jupyter]
!pip install --quiet deepspeed
!pip install --quiet openai
!pip install --quiet gradio
!pip install --quiet peft
!pip install --quiet --upgrade datasets
!pip install --quiet simplejson
!pip install --quiet accelerate
!pip install --quiet bitsandbytes

Looking in indexes: --quiet, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://test.pypi.org/simple/ (9.2 MB)
  ERROR: Cannot unpack file /tmp/pip-unpack-l0_tiun7/simple.html (downloaded from /tmp/pip-req-build-3bq6wfg8, content-type: text/html); cannot detect archive format
ERROR: Cannot determine archive format of /tmp/pip-req-build-3bq6wfg8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


##Vanilla Llama v. 8bit_Quantized Llama##

vanilla llama is slightly more capable than quantized llama, but not by much. For example, vanilla llama tends to repeat the same information less. 

In [ ]:
import os
import numpy as np
import accelerate
import bitsandbytes
import transformers
import pandas as pd
import plotly.express as px
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

from transformers import AutoTokenizer, LlamaTokenizer, LlamaForCausalLM

model = LlamaForCausalLM.from_pretrained(
            "sallywww/trained_llama_stanford_format" )

tokenizer = LlamaTokenizer.from_pretrained(
            "sallywww/trained_llama_stanford_format"
        )
#model = LlamaForCausalLM.from_pretrained('sallywww/Llama-7B')
#tokenizer = AutoTokenizer.from_pretrained('sallywww/Llama-7B')

#model.push_to_hub('sallywww/Llama-7B')
#tokenizer.push_to_hub('sallywww/Llama-7B')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:

prompt = "a loop invariant of smart contracts is " 
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=100)
tokenizer.batch_decode(generate_ids, 
                       skip_special_tokens=True, 
                       temperature=0.8, 
                       top_p=0.95, 
                       top_k=50, 
                       repeat_penalty=0.2,
                       clean_up_tokenization_spaces=False)[0]


' a loop invariant of smart contracts is  assert(x == y);'

##Inference with trained llama##

Let's make some inferences without targeted instruction finetuning yet

In [ ]:
contract = 'suicide.sol'
test_file = open(f'/content/drive/MyDrive/experiments/baseline benchmark/{contract}', "r")
test_data = test_file.read()

In [ ]:
import argparse
import random
import torch
#import gradio as gr
import torch.nn as nn
#import bitsandbytes
from transformers import LlamaForCausalLM, LlamaTokenizer

def infer_invariants(model, tokenizer, test_data, \
                     temperature, top_p, top_k,  max_new_tokens):
  model.cuda()
  model.eval()
  inputs = tokenizer(test_data, return_tensors="pt")
  with torch.no_grad():
      outputs = model.generate(input_ids=inputs["input_ids"].cuda(), \
       temperature=temperature, 
       top_p=top_p, 
       top_k=top_k, 
      max_new_tokens=max_new_tokens)
      invariants = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
  return invariants

In [ ]:
output = infer_invariants(model=model, 
                          tokenizer=tokenizer, 
                          test_data=test_data,
                          temperature=None, 
                          top_p=None, 
                          top_k=None, 
                          max_new_tokens=670)
print(output.strip())
with open(f"/content/drive/MyDrive/experiments/evaluation/noPEFT_finetuned_llama/{contract}", "w", encoding="utf-8") as file:  
    file.write(output.strip()+"\n")
file.close()

1  // SPDX-License-Identifier: MIT
2  pragma solidity >=0.4.24 <0.6.0;
3  
4  contract suicide{
5         function initOwners(address[] _owners,
6                             uint _required){
7         if (m_numOwners > 0) throw;
8         m_numOwners = _owners.length + 1;
9         m_owners[1] = uint(msg.sender);
10         m_ownerIndex[uint(msg.sender)] = 1;
11         m_required = _required;
12 
13         }
14 
15         function suicide(address _to) {
16             uint ownerIndex = m_ownerIndex[uint(msg.sender)];
17             if (ownerIndex == 0) return;
18             var pending = m_pending[sha3(msg.data)];
19             if (pending.yetNeeded == 0) {
20             pending.yetNeeded = m_required;
21             pending.ownersDone = 0;
22             }
23             uint ownerIndexBit = 2**ownerIndex;
24             if (pending.ownersDone   ownerIndexBit == 0) {
25             if (pending.yetNeeded <= 1)
26                 suicide(_to);
27             else {
28            

In [ ]:
for d in os.listdir(): 
  if os.path.isdir(d) and d.startswith('lora-'):
      print(d)
      lora_model = d
  
inference = generate_text(
                lora_param,
                test_data,
                #temperature=0.8,
                #top_p=0.95,
                #top_k=50,
                #repeat_penalty=0.02,
                max_new_tokens=750)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##evaluating quantized-insft llama7b trained with 50 epochs##

In [ ]:
lora_param = '/content/drive/MyDrive/lora-llama7b_insft_50e'
'''
for d in os.listdir(): 
     if os.path.isdir(d) and d.startswith('lora-'):
              if '7b' in d and 'insft' in d: 
                print(d)
                lora_param = d
'''

"\nfor d in os.listdir(): \n     if os.path.isdir(d) and d.startswith('lora-'):\n              if '7b' in d and 'insft' in d: \n                print(d)\n                lora_param = d\n"

In [ ]:
import argparse
import random
import torch
#import gradio as gr
import torch.nn as nn
#import bitsandbytes
from datasets import Dataset
from peft import PeftConfig, LoraConfig, get_peft_model, PeftModel
from transformers import LlamaForCausalLM, LlamaTokenizer

def load_trained_llama(model_name):
  peft_model_id = model_name
  config = PeftConfig.from_pretrained(peft_model_id)
  model = LlamaForCausalLM.from_pretrained(config.base_model_name_or_path)
  model = PeftModel.from_pretrained(model, peft_model_id)
  tokenizer = LlamaTokenizer.from_pretrained(config.base_model_name_or_path)
  model.cuda()
  return model, tokenizer
def infer_invariants(model, tokenizer, test_data, \
                     temperature, top_p, top_k,  max_new_tokens):
  model.eval()
  inputs = tokenizer(test_data, return_tensors="pt")
  with torch.no_grad():
      outputs = model.generate(input_ids=inputs["input_ids"].cuda(), \
       temperature=temperature, 
       top_p=top_p, 
       top_k=top_k, 
      max_new_tokens=max_new_tokens)
      invariants = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
  return invariants

In [ ]:
model, tokenizer = load_trained_llama(model_name=lora_param)

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│ in load_trained_llama:17                                                                         │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/torch/nn/modules/module.py:749 in cuda                    │
│                                                                                                  │
│    746 │   │   Returns:                                                                          │
│    747 │   │   │   Module: self                                                                  │
│    748 │   │   """                                                                               │
│ ❱  749 │   │   return self._apply(lambda t: t.cuda(device))                                      │
│    750 │                                                                                         │
│    751 │   def ipu(self: T, device: Optional[Union[int, device]] = None) -> T:                   │
│    752 │   │   r"""Moves all model parameters and buffers to the IPU.                            │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/torch/nn/modules/module.py:641 in _apply                  │
│                                                                                                  │
│    638 │                                                                                         │
│    639 │   def _apply(self, fn):                                                                 │
│    640 │   │   for module in self.children():                                                    │
│ ❱  641 │   │   │   module._apply(fn)                                                             │
│    642 │   │                                                                                     │
│    643 │   │   def compute_should_use_set_data(tensor, tensor_applied):                          │
│    644 │   │   │   if torch._has_compatible_shallow_copy_type(tensor, tensor_applied):           │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/torch/nn/modules/module.py:641 in _apply                  │
│                                                                                                  │
│    638 │                                                                                         │
│    639 │   def _apply(self, fn):                                                                 │
│    640 │   │   for module in self.children():                                                    │
│ ❱  641 │   │   │   module._apply(fn)                                                             │
│    642 │   │                                                                                     │
│    643 │   │   def compute_should_use_set_data(tensor, tensor_applied):                          │
│    644 │   │   │   if torch._has_compatible_shallow_copy_type(tensor, tensor_applied):           │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/torch/nn/modules/module.py:641 in _apply                  │
│                                                                                                  │
│    638 │                                                                                         │
│    639 │   def _apply(self, fn):                                                                 │
│    640 │   │   for module in self.children():                                                    │
│ ❱  641 │   │   │   module._apply(fn)                       

In [ ]:
contract = 'Replica.sol'
test_file = open(f'/content/drive/MyDrive/experiments/baseline benchmark/{contract}', "r")
test_data = test_file.read()

In [ ]:
output = infer_invariants(model=model, tokenizer=tokenizer, test_data=test_data, max_new_tokens=512)
print(output.strip())
with open(f"/content/drive/MyDrive/experiments/evaluation/quantized_llama_inferred_invariants/{contract}", "w", encoding="utf-8") as file:  
    file.write(output.strip()+"\n")
file.close()